Tutorial Key Features  
Session-based Context Management: Understanding how AgentCore Runtime maintains context within sessions  
Runtime Session Lifecycle: Learning about session creation, maintenance, and termination  
Context Object Access: Accessing runtime information like session ID through the context parameter  
Session Isolation: Demonstrating how different sessions provide complete isolation  
Payload Handling: Flexible data passing through custom payload structures  
Cross-invocation State: Maintaining agent state across multiple calls within the same session  

The context object in AgentCore Runtime provides valuable information about the current execution environment:

session_id: The current runtime session identifier  
Runtime Metadata: Information about the runtime environment  
Execution Details: Context about the current invocation

In [ ]:
%%writefile strands_claude_context.py
from strands import Agent, tool
from strands_tools import calculator # Import the calculator tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import asyncio
from datetime import datetime

app = BedrockAgentCoreApp()

# Create a custom tool 
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@tool
def get_time():
    """ Get current time """
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[
        calculator, weather, get_time
    ],
    system_prompt="""
    You're a helpful assistant. You can do simple math calculations, 
    tell the weather, and provide the current time.
    Always start by acknowledging the user's name 
    """
)

def get_user_name(user_id):
    users = {
        "1": "Maira",
        "2": "Mani",
        "3": "Mark",
        "4": "Ishan",
        "5": "Dhawal"
    }
    return users[user_id]
    
@app.entrypoint
def strands_agent_bedrock_handling_context(payload, context):
    """
    AgentCore Runtime entrypoint that demonstrates context handling and session management.
    
    Args:
        payload: The input payload containing user data and request information
        context: The runtime context object containing session and execution information
    
    Returns:
        str: The agent's response incorporating context information
    """
    user_input = payload.get("prompt")
    user_id = payload.get("user_id")
    user_name = get_user_name(user_id)
    
    # Access runtime context information
    print("=== Runtime Context Information ===")
    print("User id:", user_id)
    print("User Name:", user_name)
    print("User input:", user_input)
    print("Runtime Session ID:", context.session_id)
    print("Context Object Type:", type(context))
    print("=== End Context Information ===")
    
    # Create a personalized prompt that includes context information
    prompt = f"""My name is {user_name}. Here is my request: {user_input}
    
    Additional context: This is session {context.session_id}. 
    Please acknowledge my name and provide assistance."""
    
    response = agent(prompt)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

In [1]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude_context.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    requirements_file="pyproject.toml",
    region=region,
    agent_name="strands_claude_context"
)

Entrypoint parsed: file=/home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/runtime-context/strands_claude_context.py, bedrock_agentcore_name=strands_claude_context
Configuring BedrockAgentCore agent: strands_claude_context


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated .dockerignore
Generated Dockerfile: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/runtime-context/Dockerfile
Generated .dockerignore: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/runtime-context/.dockerignore
Setting 'strands_claude_context' as default agent
Bedrock AgentCore configured: /home/saeid/dev/github/gen-ai/agentcore/bedrock-agentcore-samples/runtime-context/.bedrock_agentcore.yaml


In [3]:
launch_result = agentcore_runtime.launch()

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_claude_context' to account 381491838394 (ap-southeast-2)
Setting up AWS resources (ECR repository, execution roles)...
Using ECR repository from config: 381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude_context
Using execution role from config: arn:aws:iam::381491838394:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-2-969e6d3fbf
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_claude_context
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-

In [5]:
import uuid
import json
from IPython.display import Markdown, display

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()
print(f"📋 Starting Session 1: {session_id}")
print(f"👤 User: Maira (ID: 1)")
print(f"❓ First question about weather\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather outside?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

📋 Starting Session 1: f7e2bd8b-b340-4b16-90dd-020c67103d51
👤 User: Maira (ID: 1)
❓ First question about weather



Great news, Maira! The weather outside is **sunny**. It looks like a beautiful day! ☀️

Is there anything else you'd like to know, such as the current time or any math calculations?

In [6]:
# Continue with the same session ID to demonstrate session continuity
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Follow-up question about math\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How much is 2X5?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🔄 Continuing Session 1: f7e2bd8b-b340-4b16-90dd-020c67103d51
👤 Same user: Maira (ID: 1)
❓ Follow-up question about math



The answer is **10**, Maira! 

2 × 5 = 10

Is there anything else you'd like me to calculate or help you with?

In [7]:
# Continue with the same session ID - notice how the agent remembers the previous calculation
print(f"🔄 Continuing Session 1: {session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Building on previous answer - demonstrates context continuity\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "and that plus 34?",
    "user_id": "1"
}, session_id=str(session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🔄 Continuing Session 1: f7e2bd8b-b340-4b16-90dd-020c67103d51
👤 Same user: Maira (ID: 1)
❓ Building on previous answer - demonstrates context continuity



The answer is **44**, Maira!

10 + 34 = 44

So to summarize: 2 × 5 = 10, and then 10 + 34 = 44. Is there anything else you'd like me to help with?

In [8]:
# NEW SESSION - Demonstrate session isolation
# Create a completely new session ID to show that context is lost
new_session_id = uuid.uuid4()
print(f"🆕 Starting NEW Session 2: {new_session_id}")
print(f"👤 Same user: Maira (ID: 1)")
print(f"❓ Attempting to reference previous calculation - should fail due to session isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "And plus 10?",
    "user_id": "1"
}, session_id=str(new_session_id))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🆕 Starting NEW Session 2: cd0b9ad0-32b4-4bd1-8e94-e1ea75fc47e6
👤 Same user: Maira (ID: 1)
❓ Attempting to reference previous calculation - should fail due to session isolation



Hello Maira! Nice to meet you! 👋

I'd be happy to help you with your math calculation. However, I need a bit more information - you've asked me to "add plus 10" but I need to know what number you'd like to add 10 to.

Could you please provide the starting number? For example, are you looking to calculate something like:
- 5 + 10?
- 20 + 10?
- Or perhaps a different number?

Once you let me know, I'll calculate it for you right away!

In [9]:
# NEW SESSION AND USER - Demonstrate complete isolation
different_user_session = uuid.uuid4()
print(f"🆕 Starting Session 3: {different_user_session}")
print(f"👤 Different user: Mani (ID: 2)")
print(f"❓ Same question as first user - demonstrates user isolation\n")

invoke_response = agentcore_runtime.invoke({
    "prompt": "How is the weather?",
    "user_id": "2"
}, session_id=str(different_user_session))

response_data = invoke_response['response'][0]
display(Markdown(response_data))

🆕 Starting Session 3: 8a451d93-b377-4a18-909a-7032063264b1
👤 Different user: Mani (ID: 2)
❓ Same question as first user - demonstrates user isolation



The weather is looking great, Mani! ☀️ It's **sunny** outside. Perfect weather for being outdoors! Is there anything else you'd like to know?

In Session 1 The agent maintained context across multiple invocations within the same session, remembering the calculation result from the previous interaction.   
In session 2 Even with the same user, a new session creates a completely isolated environment with no access to previous context.  
In session 3 Each session provides complete isolation, ensuring privacy and security between different users and interactions.

In [10]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

('381491838394.dkr.ecr.ap-southeast-2.amazonaws.com/bedrock-agentcore-strands_claude_context',
 'strands_claude_context-T3mH414146',
 'bedrock-agentcore-strands_claude_context')

In [11]:
import boto3

agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
)

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)